### Index

In [8]:
import re
import gc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter
from math import log
import numpy as np

text_filename = './data/cran.all.1400'
query_filename = './data/cran.qry'

with open(text_filename,'r') as f:
    text = f.read()
with open(query_filename,'r') as f:
    queries = f.read()

    
def clearWPart(s, source, target):
    ind = s.find(source) + len(source)
    return s[:ind] + s[ind:].replace(source,target)
    
    
def text_normalization(dirty_text):
    clear_regexp = r"\W+"
    c_words = re.sub(clear_regexp, ' ', dirty_text, flags=re.DOTALL).strip().split()
    stemmer = PorterStemmer()
    filtered_words = [word for word in c_words if word not in stopwords.words('english')]
    return [stemmer.stem(i) for i in filtered_words]


def parse_text(text):
    regex = r"^\.[ITW](.|\n)*?(?=\n\.[AB]|\Z)"
    matches = re.finditer(regex, text, re.MULTILINE)
    f_text = ' '.join([e.group() for e in matches])
    f_text = '.I'.join([clearWPart(i,'.W','') for i in f_text.split('.I')])
    docs = re.split('.I|.T|.W', f_text)
    t_docs = [[docs[i], docs[i + 1], docs[i + 2]] for i in xrange(1, len(docs)-2, 3)]
    
    c_docs = map(lambda r: (int(r[0].strip(' \t\n\r')),\
                       text_normalization(r[1]),\
                       text_normalization(r[2])),\
            t_docs)
    return c_docs


def parse_queries(queries):
    qrs = re.split('.I|.W', queries)
    t_qrs = [[qrs[i], qrs[i + 1]] for i in xrange(1, len(qrs)-1, 2)]
    c_qrs = map(lambda r: (int(r[0].strip(' \t\n\r')),text_normalization(r[1])),t_qrs)
    return c_qrs


docs = parse_text(text)
qrs = parse_queries(queries)


def create_ivn_index(data, part='doc'):
    idx = 3 if part == 'doc' else 1
    wc_data = map(lambda r: (r[0],  [(k,v) for k,v in Counter(r[1]).iteritems()],len(r[1]),\
                                    [(k,v) for k,v in Counter(r[2]).iteritems()],len(r[2])),
                  data)
    p_data =  map(lambda r: zip([r[0]]*len(r[idx]),
                                r[idx],
                                [r[idx + 1]]*len(r[idx])), 
                  wc_data)
    
    flatted_data = [item for sublist in p_data for item in sublist] 
    index = defaultdict(list)
    for k, v, doc_l in flatted_data:
        index[v[0]].append((k,v[1],doc_l))
    index['__metadata__'] = {'num_of_docs':len(data),
                             'index_len':len(index),
                             'avgdl':np.average([len(docs[idx]) for docs in wc_data]),
                             'avg_docs_len':np.average([len(v) for k,v in index.iteritems()]),
                             'max_docs_len':np.max([len(v) for k,v in index.iteritems()])}
    return index


doc_index = create_ivn_index(docs)
print 'document index metadata: '
doc_index['__metadata__']
title_index = create_ivn_index(docs, part='title')
print 'title index metadata: '
title_index['__metadata__']

document index metadata: 


{'avg_docs_len': 18.416861329369294,
 'avgdl': 61.946428571428569,
 'index_len': 4709,
 'max_docs_len': 730,
 'num_of_docs': 1400}

title index metadata: 


{'avg_docs_len': 8.5413819286256647,
 'avgdl': 8.0350000000000001,
 'index_len': 1317,
 'max_docs_len': 365,
 'num_of_docs': 1400}

### Search

In [9]:
def process_query(index, query):
    
    def score_BM25(N, nq, ft, D, avgdl):
        k1 = 1.2
        b = 0.75
        #k2 = 0.01
        def IDF(N, nq):
            return log(1 + (N - nq + 0.5)/(nq + 0.5))
    
        return IDF(N,nq) * (ft*(k1 + 1))/(ft + k1*(1 - b + b * (D/avgdl))) #* ((k2 + 1)*ft/(k2 + ft)

    num_docs_in_coll = index['__metadata__']['num_of_docs']
    avgdl = index['__metadata__']['avgdl']
    
    query_result = dict()
    for term in query:
        if term in index:
            term_docs = index[term]
            for docid, freq, doc_len in term_docs:
                score = score_BM25(N=num_docs_in_coll, nq=len(term_docs),ft=freq,D=doc_len, avgdl=avgdl)
                if docid in query_result:
                    query_result[docid] += score
                else:
                    query_result[docid] = score
    return query_result


def search_queries(index, queries, file_name=None):
    if file_name:
        f = open(file_name,'w')
    for i,q in enumerate(queries):
        query_res = process_query(index, q[1])
        for doc_id in sorted(query_res, key=query_res.get, reverse=True)[:10]:
            if file_name:
                f.write(str(i+1) + ' ' + str(doc_id) + '\n')
            else:
                print i + 1, doc_id
    if file_name:
        f.close()

        
search_queries(doc_index, qrs, file_name='./data/answer_doc')
search_queries(title_index, qrs, file_name='./data/answer_title')

### Results evaluation

In [10]:
groundtruth_file = './data/test.qrel_clean'
title_answer_file = './data/answer_title'
doc_answer_file = './data/answer_doc'

def res_eval(groundtruth_file, answer_file):
    q2reld = {} 
    for line in open(groundtruth_file):
        qid, did = [int(x) for x in line.split()]
        if qid in q2reld.keys():
            q2reld[qid].add(did)
        else:
            q2reld[qid] = set()

    q2retrd = {}
    for line in open(answer_file):
        qid, did = [int(x) for x in line.split()]
        if qid in q2retrd.keys():
            q2retrd[qid].append(did)
        else:
            q2retrd[qid] = []    


    N = len(q2retrd.keys())
    precision = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2retrd[q]) for q in q2retrd.keys()]) / N
    recall = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2reld[q]) for q in q2retrd.keys()]) / N
    print "mean precision: {}\nmean recall: {}\nmean F-measure: {}".format(precision, recall, 2*precision*recall/(precision+recall)) 

    # MAP@10
    import numpy as np

    MAP = 0.0
    for q in q2retrd.keys():
        n_results = min(10, len(q2retrd[q]))
        avep = np.zeros(n_results)
        for i in range(n_results):
            avep[i:] += q2retrd[q][i] in q2reld[q]
            avep[i] *= (q2retrd[q][i] in q2reld[q]) / (i+1.0)
        MAP += sum(avep) / min(n_results, len(q2reld[q]))
    print "MAP@10: {}".format(MAP/N) 

print 'search by docs: '
res_eval(groundtruth_file,doc_answer_file)
print '\n'
print 'search by titles: '
res_eval(groundtruth_file,title_answer_file)

search by docs: 
mean precision: 0.22024691358
mean recall: 0.306024492383
mean F-measure: 0.256145210109
MAP@10: 0.21955973307


search by titles: 
mean precision: 0.173333333333
mean recall: 0.246569763465
mean F-measure: 0.203564866877
MAP@10: 0.170676459926
